In [1]:
%load_ext autoreload
%autoreload 2

## How to scrape any website with ScraperAI

### Before we start, install the package

In [2]:
# ! pip install scraperai

In [10]:
import datetime
import json
import os

import click
import pandas as pd
from dotenv import find_dotenv, load_dotenv

from scraperai import BaseCrawler
from scraperai.cli.model import ScreenStatus
from scraperai.cli.view import View
from scraperai.exceptions import NotFoundError
from scraperai.parsers import WebpageType, Pagination
from scraperai.parsers.models import CatalogItem, WebpageFields
from scraperai.scraper import ScraperAI, ScrapingSummary
from scraperai.crawlers import SeleniumCrawler

### Step 1. Init crawler

We are opening the new browser window for the next steps. Driver will be automatically installed on your machine. We use Chrome as default browser. If you want to use another browser, you need to create another implementation of the `BaseCrawler` class.

In [11]:
# TODO: Check opened crawlers before start a new one. Maybe, singletone pattern?
crawler = SeleniumCrawler()

### Step 2. Init AI Scraper
By default, we are using OpenAI GPT-4. You can place your API key in the `.env` file. If you don't have it, you can get it [here](https://platform.openai.com/api-keys).
Also, you can use another AI model. Just create another implementation of the `BaseLM` and `BaseVision` class.

In [12]:
env_file = find_dotenv()
if env_file:
    load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key is None:
    openai_api_key = input('Please, enter your OpenAI API key: ')
    # TODO: save to .env

scraper = ScraperAI(crawler=crawler, openai_api_key=openai_api_key)

There are 2 experiments in this doc:
1. [List of YCombinator companies](https://www.ycombinator.com/companies/)
2. [List of commits in the repository](https://github.com/scraperai/scraperai/commits/main/)

#### Experiment 1. List of YCombinator companies

### Step 3. Open the website page
Later, in case of multiple similar sites you will be able to run batch scraping. The main target is to semi-automatically detect the all xpaths

In [13]:
URL = 'https://www.ycombinator.com/companies' # Enter the URL of the website

In [14]:
# Open page in the browser
scraper.crawler.get(URL)

### Step 3.1. Detecting the page type
Take a screenshot and detect a type using Vision Model (GPT4 Vision by default). 
If you know the type of the page, you can set it manually. There are two types of pages: `catalog` and `detailed_page`
Typically, we are working with catalog items. It can be a list of products, articles, companies, table rows, etc.

In [15]:
# page_type = scraper.detect_page_type(URL)
page_type = WebpageType.CATALOG 
page_type

<WebpageType.CATALOG: 'catalog'>

### Step 3.2. Detecting the pagination
**It is used only for type `catalog`.**
We need to pass a whole page to detect the pagination. 
There 3 types of pagination: `xpath`, `scroll`, and `url_param` (not implemented yet).
In case of error, you can set it manually.

In [16]:
# pagination = scraper.detect_pagination(URL)
pagination = Pagination(type='scroll')
pagination

Pagination(type='scroll', xpath=None, url_param=None)

### Step 3.3. Detecting the catalog items
**It is used only for type `catalog`.**
You should correctly choose item block, url and ... 
AI isn't perfect, so you can manually add extra prompt to help AI to understand what you want or set xpath manually.

In [17]:
try:
    catalog_item = scraper.detect_catalog_item(URL, extra_prompt = None)
except NotFoundError:
    catalog_item = None
catalog_item

ParserError: Document is empty

In [ ]:
# TODO: continue with this step. Stopped on the error "Document is empty"

In [18]:
html_content = """
<html><head><title>The YC Startup Directory | Y Combinator</title><link href="/assets/ycdc/favicon-c8a914eeeba9fe6f7a863b35608b55aeedd7c1ff409c97b9ecb96b7a6c278d70.ico"/><link href="/assets/ycdc/favicon-c8a914eeeba9fe6f7a863b35608b55aeedd7c1ff409c97b9ecb96b7a6c278d70.ico"/><link href="https://www.ycombinator.com/blog/feed"/><link href="https://plus.google.com/113116812547402050884"/><link href="/vite/assets/YCAlerts.module-tk5ZRkuS.js"/><link href="/vite/assets/BookmarkButton.module-GdGbqxXk.js"/><link href="/vite/assets/styles.module-LI75PDVS.js"/><link href="/vite/assets/FormBuilder.module-onQP7rom.js"/><link href="/vite/assets/SharedStyles.module-PSUmCxTq.js"/><link href="/vite/assets/index-dyyTtTr9.js"/><link href="/vite/assets/component_registry-6joeGzJX.js"/><link href="/vite/assets/index-7dv89_Cl.js"/><link href="/vite/assets/_commonjsHelpers-4gQjN7DL.js"/><link href="/vite/assets/index.esm-McsrC-9X.js"/><link href="/vite/assets/index-gDwDDouJ.js"/><link href="/vite/assets/withRailsContext-qEVvX3Q4.js"/><link href="/vite/assets/___vite-browser-external_commonjs-proxy-LJ3sMCxY.js"/><link href="/vite/assets/BookfaceCsrApp-0EsRinPR.js"/><link href="/vite/assets/index-5mw1yDQK.js"/><link href="/vite/assets/PostShowPage-CUfyWBvX.js"/><link href="/vite/assets/moment-QXvGtVow.js"/><link href="/vite/assets/QueryStringUtils-bCJlS7Nh.js"/><link href="/vite/assets/_getPrototype-4y_d_TY_.js"/><link href="/vite/assets/amplitude-twPdXQG1.js"/><link href="/vite/assets/authorizedFetch-uEYPuZPU.js"/><link href="/vite/assets/renderMarkdown-xyeZHd99.js"/><link href="/vite/assets/lodash-gzw5KmHd.js"/><link href="/vite/assets/util-IAp02Z3C.js"/><link href="/vite/assets/debounce-kuzkKLms.js"/><link href="/vite/assets/index.es-qoqTidCB.js"/><link href="/vite/assets/index-VFMbf6KQ.js"/><link href="/vite/assets/index-3uQdc6nR.js"/><link href="/vite/assets/index-VJlLTEI1.js"/><link href="/vite/assets/Modal-lhjy-Vf1.js"/><link href="/vite/assets/cn-UGDbM6oy.js"/><link href="/vite/assets/toConsumableArray-Qns-LOUw.js"/><link href="/vite/assets/index-MrRY6v07.js"/><link href="/vite/assets/createLucideIcon-UI5OVsP-.js"/><link href="/vite/assets/index-s7oKE3jZ.js"/><link href="/vite/assets/memoize-one.esm-BXZKDq4t.js"/><link href="/vite/assets/v4-McvpxQtQ.js"/><link href="/vite/assets/MarkdownEditor-BqoQit0E.js"/><link href="/vite/assets/ResizeObserver.es-Ip45Sxpy.js"/><link href="/vite/assets/algoliaClient-BHW9TlVB.js"/><link href="/vite/assets/algoliasearch-lWFc5VoW.js"/><link href="/vite/assets/index-UwPYAqGP.js"/><link href="/vite/assets/_baseForOwn-0vfXj26N.js"/><link href="/vite/assets/formik.esm-sLVlTXya.js"/><link href="/vite/assets/immutable.es-UU2iDYKb.js"/><link href="/vite/assets/TimeAgo-aPM0fvBM.js"/><link href="/vite/assets/Select-7TJFkqGt.js"/><link href="/vite/assets/index-Cezb-nEm.js"/><link href="/vite/assets/algoliasearch-lite.umd-xZtPrxHb.js"/><link href="/vite/assets/YCAlerts-_qb2PGC5.js"/><link href="/vite/assets/index-f7i4c01P.js"/><link href="/vite/assets/Helmet-0hJ1wLbC.js"/><link href="/vite/assets/ThrottledInfiniteScroll-2y5TtQp8.js"/><link href="/vite/assets/throttle-dkcnUqCJ.js"/><link href="/vite/assets/Alert-Fh0pTerO.js"/><link href="/vite/assets/createSvgIcon-nZEM8JCk.js"/><link href="/vite/assets/CopyToClipboard-NNMtFaJU.js"/><link href="/vite/assets/SharedDirectory-IfCxqcC6.js"/><link href="/vite/assets/react-datepicker-rEY4tApA.js"/><link href="/vite/assets/jquery-_-XaHmv-.js"/><link href="/vite/assets/utils-l6miV6cT.js"/><link href="/vite/assets/index-4bJ7FMLj.js"/><link href="/vite/assets/Tabs-HiqAPXfc.js"/><link href="/vite/assets/Pills-6U7M8Bas.js"/><link href="/vite/assets/CompanyListItemContent-sP4oVR1O.js"/><link href="/vite/assets/utils-vlqiQ9eD.js"/><link href="/vite/assets/preload-helper-YBYoqhTB.js"/><link href="/vite/assets/ycdc-styles-6V9kbd80.css"/><link href="/vite/assets/Launches-z5F32Efs.css"/><link href="/vite/assets/LaunchesFormPage-oiGvgDPe.css"/><link href="/vite/assets/LaunchesIndexPage-hAPAx7AJ.css"/><link href="/vite/assets/LaunchesPage-LdxYw7ET.css"/><link href="/vite/assets/LaunchesShowPage-BLQHJuqg.css"/><link href="/vite/assets/WhyPage-sDAepCfX.css"/><link href="/vite/assets/Breadcrumb-EPIwYLCu.css"/><link href="/vite/assets/nouislider-hDW9_Shu.css"/><link href="/vite/assets/editor-CkOJ1I6z.css"/><link href="/vite/assets/Dropdown-_91FvgQD.css"/><link href="/vite/assets/tooltip-A79TiECk.css"/><link href="/vite/assets/index-syxAMI0e.css"/><link href="/vite/assets/YCAlerts-wo845qFC.css"/><link href="/vite/assets/BookmarkButton-mP9K9-X2.css"/><link href="/vite/assets/styles-T_QGThXq.css"/><link href="/vite/assets/FormBuilder-070RcrQG.css"/><link href="/vite/assets/SharedStyles-Tp6e4CeA.css"/><link href="/vite/assets/index-TlqxYGig.css"/><link href="/vite/assets/BookfaceCsrApp-HkPQuzev.css"/><link href="/vite/assets/react-datepicker-0M4vIhpG.css"/><link href="/vite/assets/MarkdownEditor-UvangTfq.css"/><link href="/vite/assets/SharedDirectory-QFQwagkg.css"/><link href="/vite/assets/IndexPage-Hnt_vRTQ.js"/><link href="/vite/assets/Section-iu3N0lG4.js"/><link href="/vite/assets/XIcon-yfAkV1_o.js"/><link href="/vite/assets/use-root-containers-7-G-Ilt6.js"/><link href="/vite/assets/transition-sJatrXft.js"/><link href="/vite/assets/link_component-AMXQBkDt.js"/><link href="/vite/assets/HorizontalLinks-CSCh2s4A.js"/><link href="/vite/assets/YcdcCompanyDirectory-RU_lDVuI.js"/><link href="/vite/assets/ResponsiveProse-fteiIl2t.js"/></head><body class="retro ycdc2 companies index"><div><div class="no-scroll-behind relative isolate z-10"><div></div><div></div></div><div class="mx-auto max-w-ycdc-page"><section class="relative isolate z-0 border-retro-sectionBorder sm:pr-[13px] ycdcPlus:pr-0 border-b pt-6 lg:pt-9 pb-6 lg:pb-9"><div class="prose prose-sm sm:prose-base max-w-full"><h1>Startup Directory</h1><p>Since 2005, we have invested in over 4,000 companies that have a combined valuation of over $600B.</p><p>In this directory, you can search for YC companies by industry, region, company size, and more.</p><p>To find jobs at these startups, visit <a href="/jobs">Work at a Startup</a>.</p><p>At the links below you'll find directories of companies started by historically underrepresented founders. For demographic data on our batches, visit <a href="/blog/tag/batch-stats">our blog</a>.</p><p class="mt-4 flex flex-col space-y-3 pl-3 font-medium text-linkColor sm:flex-row sm:space-y-0"><a class="mr-4 inline-flex flex-nowrap items-center text-base" href="/companies/black-founders"><span>Black-founded companies</span><svg class="ml-1 inline h-4 w-4 md:h-5 md:w-5"><path></path></svg></a><a class="mr-4 inline-flex flex-nowrap items-center text-base" href="/companies/hispanic-latino-founders"><span>Hispanic &amp; Latino-founded companies</span><svg class="ml-1 inline h-4 w-4 md:h-5 md:w-5"><path></path></svg></a><a class="mr-4 inline-flex flex-nowrap items-center text-base" href="/companies/women-founders"><span>Women-founded companies</span><svg class="ml-1 inline h-4 w-4 md:h-5 md:w-5"><path></path></svg></a></p></div></section><section class="relative isolate z-0 border-retro-sectionBorder sm:pr-[13px] ycdcPlus:pr-0 border-b pt-6 lg:pt-9 pb-6 lg:pb-9 ycdc-with-link-color"><div class="space-y-2"><div class="flex justify-end"><div class="flex space-x-2"><label class="mb-auto mt-auto whitespace-nowrap">Sort by</label><select class="w-full rounded-md border-gray-300 pl-3 pr-10 pr-6 text-base focus:border-indigo-500 focus:outline-none focus:ring-indigo-500 sm:text-sm"><option>Default</option><option>Launch Date</option></select></div></div><div class="_sharedDirectory_19sud_76"><div class="_leftCol_19sud_131"><div class="_section_19sud_146 _facets_19sud_200"><div class="_facet_19sud_85"><label><input/><span class="_label_19sud_225">💎 Top Companies by Revenue</span><span class="_count_19sud_291">50</span></label></div><div class="_facet_19sud_85"><label><input/><span class="_label_19sud_225">💎 Top Companies by Valuation</span><span class="_count_19sud_291">341</span></label></div><div class="_facet_19sud_85"><label><input/><span class="_label_19sud_225">Is Hiring</span><span class="_count_19sud_291">975</span></label></div><div class="_facet_19sud_85"><label><input/><span class="_label_19sud_225">Nonprofit</span><span class="_count_19sud_291">43</span></label></div><div class="_line_19sud_215"></div><div class="_facet_19sud_85"><label><input/><span class="_label_19sud_225">Black-founded</span><span class="_count_19sud_291">260</span></label></div><div class="_facet_19sud_85"><label><input/><span class="_label_19sud_225">Hispanic &amp; Latino-founded</span><span class="_count_19sud_291">463</span></label></div><div class="_facet_19sud_85"><label><input/><span class="_label_19sud_225">Women-founded</span><span class="_count_19sud_291">696</span></label></div><div class="_line_19sud_215"></div><div class="_facet_19sud_85"><h4>Batch </h4><div class="_collapseToggle_19sud_257"><svg class="svg-inline--fa fa-square-minus"><path></path></svg></div><label><input/><span class="_label_19sud_225">All batches</span><span class="_count_19sud_291">4646</span></label><div><label><input/><span class="_label_19sud_225">W24</span><span class="_count_19sud_291">229</span></label></div><div><label><input/><span class="_label_19sud_225">S23</span><span class="_count_19sud_291">220</span></label></div><div><label><input/><span class="_label_19sud_225">W23</span><span class="_count_19sud_291">273</span></label></div><div><label><input/><span class="_label_19sud_225">S22</span><span class="_count_19sud_291">236</span></label></div><div><label><input/><span class="_label_19sud_225">W22</span><span class="_count_19sud_291">403</span></label></div><div><label><input/><span class="_label_19sud_225">S21</span><span class="_count_19sud_291">392</span></label></div><div><label><input/><span class="_label_19sud_225">W21</span><span class="_count_19sud_291">337</span></label></div><a class="_showMoreLess_19sud_241" href="#">See all options</a></div><div class="_line_19sud_215"></div><div class="_facet_19sud_85"><h4>Industry </h4><div class="_collapseToggle_19sud_257"><svg class="svg-inline--fa fa-square-minus"><path></path></svg></div><label><input/><span class="_label_19sud_225">All industries</span><span class="_count_19sud_291">4646</span></label><div><label><input/><button class="_plain_19sud_125"><svg class="svg-inline--fa fa-caret-right fa-fw _expandFacet_19sud_303"><path></path></svg></button><span class="_label_19sud_225">B2B</span><span class="_count_19sud_291">2153</span></label></div><div><label><input/><span class="_label_19sud_225">Education</span><span class="_count_19sud_291">187</span></label></div><div><label><input/><button class="_plain_19sud_125"><svg class="svg-inline--fa fa-caret-right fa-fw _expandFacet_19sud_303"><path></path></svg></button><span class="_label_19sud_225">Fintech</span><span class="_count_19sud_291">553</span></label></div><div><label><input/><button class="_plain_19sud_125"><svg class="svg-inline--fa fa-caret-right fa-fw _expandFacet_19sud_303"><path></path></svg></button><span class="_label_19sud_225">Consumer</span><span class="_count_19sud_291">783</span></label></div><div><label><input/><button class="_plain_19sud_125"><svg class="svg-inline--fa fa-caret-right fa-fw _expandFacet_19sud_303"><path></path></svg></button><span class="_label_19sud_225">Healthcare</span><span class="_count_19sud_291">553</span></label></div><div><label><input/><button class="_plain_19sud_125"><svg class="svg-inline--fa fa-caret-right fa-fw _expandFacet_19sud_303"><path></path></svg></button><span class="_label_19sud_225">Real Estate and Construction</span><span class="_count_19sud_291">130</span></label></div><div><label><input/><button class="_plain_19sud_125"><svg class="svg-inline--fa fa-caret-right fa-fw _expandFacet_19sud_303"><path></path></svg></button><span class="_label_19sud_225">Industrials</span><span class="_count_19sud_291">242</span></label></div><a class="_showMoreLess_19sud_241" href="#">See all options</a></div><div class="_line_19sud_215"></div><div class="_facet_19sud_85"><h4>Region </h4><div class="_collapseToggle_19sud_257"><svg class="svg-inline--fa fa-square-minus"><path></path></svg></div><label><input/><span class="_label_19sud_225">Anywhere</span><span class="_count_19sud_291">4646</span></label><div><label><input/><button class="_plain_19sud_125"><svg class="svg-inline--fa fa-caret-right fa-fw _expandFacet_19sud_303"><path></path></svg></button><span class="_label_19sud_225">America / Canada</span><span class="_count_19sud_291">3377</span></label></div><div><label><input/><button class="_plain_19sud_125"><svg class="svg-inline--fa fa-caret-right fa-fw _expandFacet_19sud_303"><path></path></svg></button><span class="_label_19sud_225">Remote</span><span class="_count_19sud_291">2806</span></label></div><div><label><input/><button class="_plain_19sud_125"><svg class="svg-inline--fa fa-caret-right fa-fw _expandFacet_19sud_303"><path></path></svg></button><span class="_label_19sud_225">Europe</span><span class="_count_19sud_291">403</span></label></div><div><label><input/><button class="_plain_19sud_125"><svg class="svg-inline--fa fa-caret-right fa-fw _expandFacet_19sud_303"><path></path></svg></button><span class="_label_19sud_225">South Asia</span><span class="_count_19sud_291">239</span></label></div><div><label><input/><button class="_plain_19sud_125"><svg class="svg-inline--fa fa-caret-right fa-fw _expandFacet_19sud_303"><path></path></svg></button><span class="_label_19sud_225">Latin America</span><span class="_count_19sud_291">231</span></label></div><div><label><input/><button class="_plain_19sud_125"><svg class="svg-inline--fa fa-caret-right fa-fw _expandFacet_19sud_303"><path></path></svg></button><span class="_label_19sud_225">Southeast Asia</span><span class="_count_19sud_291">111</span></label></div><div><label><input/><button class="_plain_19sud_125"><svg class="svg-inline--fa fa-caret-right fa-fw _expandFacet_19sud_303"><path></path></svg></button><span class="_label_19sud_225">Africa</span><span class="_count_19sud_291">91</span></label></div><div><label><input/><button class="_plain_19sud_125"><svg class="svg-inline--fa fa-caret-right fa-fw _expandFacet_19sud_303"><path></path></svg></button><span class="_label_19sud_225">Middle East and North Africa</span><span class="_count_19sud_291">80</span></label></div><div><label><input/><button class="_plain_19sud_125"><svg class="svg-inline--fa fa-caret-right fa-fw _expandFacet_19sud_303"><path></path></svg></button><span class="_label_19sud_225">East Asia</span><span class="_count_19sud_291">30</span></label></div><div><label><input/><button class="_plain_19sud_125"><svg class="svg-inline--fa fa-caret-right fa-fw _expandFacet_19sud_303"><path></path></svg></button><span class="_label_19sud_225">Oceania</span><span class="_count_19sud_291">17</span></label></div></div><div class="_line_19sud_215"></div><div class="_facet_19sud_85"><h4>Tags </h4><div class="_collapseToggle_19sud_257"><svg class="svg-inline--fa fa-square-minus"><path></path></svg></div><input/><div class="_typeaheadResults_19sud_632"><label><input/><span class="_label_19sud_225 group relative">SaaS<span class="_count_19sud_291">1017</span></span></label><label><input/><span class="_label_19sud_225 group relative">B2B<span class="_count_19sud_291">895</span></span></label><label><input/><span class="_label_19sud_225 group relative">Fintech<span class="_count_19sud_291">678</span></span></label><label><input/><span class="_label_19sud_225 group relative">Artificial Intelligence<span class="_count_19sud_291">456</span></span></label><label><input/><span class="_label_19sud_225 group relative">Developer Tools<span class="_count_19sud_291">438</span></span></label><label><input/><span class="_label_19sud_225 group relative">AI<span class="_count_19sud_291">319</span></span></label><label><input/><span class="_label_19sud_225 group relative">Marketplace<span class="_count_19sud_291">319</span></span></label><label><input/><span class="_label_19sud_225 group relative">E-commerce<span class="_count_19sud_291">222</span></span></label><label><input/><span class="_label_19sud_225 group relative">Generative AI<span class="_count_19sud_291">216</span></span></label><label><input/><span class="_label_19sud_225 group relative">Machine Learning<span class="_count_19sud_291">209</span></span></label><label><input/><span class="_label_19sud_225 group relative">Analytics<span class="_count_19sud_291">169</span></span></label><label><input/><span class="_label_19sud_225 group relative">Education<span class="_count_19sud_291">168</span></span></label><label><input/><span class="_label_19sud_225 group relative">Consumer<span class="_count_19sud_291">159</span></span></label><label><input/><span class="_label_19sud_225 group relative">Climate<span class="_count_19sud_291">151</span></span></label><label><input/><span class="_label_19sud_225 group relative">Healthcare<span class="_count_19sud_291">151</span></span></label><label><input/><span class="_label_19sud_225 group relative">API<span class="_count_19sud_291">145</span></span></label><label><input/><span class="_label_19sud_225 group relative">Open Source<span class="_count_19sud_291">141</span></span></label><label><input/><span class="_label_19sud_225 group relative">Health Tech<span class="_count_19sud_291">137</span></span></label><label><input/><span class="_label_19sud_225 group relative">Productivity<span class="_count_19sud_291">132</span></span></label><label><input/><span class="_label_19sud_225 group relative">Payments<span class="_count_19sud_291">128</span></span></label><label><input/><span class="_label_19sud_225 group relative">Biotech<span class="_count_19sud_291">110</span></span></label><label><input/><span class="_label_19sud_225 group relative">Consumer Health Services<span class="_count_19sud_291">104</span></span></label><label><input/><span class="_label_19sud_225 group relative">Enterprise<span class="_count_19sud_291">101</span></span></label><label><input/><span class="_label_19sud_225 group relative">Logistics<span class="_count_19sud_291">101</span></span></label><label><input/><span class="_label_19sud_225 group relative">Hardware<span class="_count_19sud_291">94</span></span></label><label><input/><span class="_label_19sud_225 group relative">Crypto / Web3<span class="_count_19sud_291">92</span></span></label><label><input/><span class="_label_19sud_225 group relative">Data Engineering<span class="_count_19sud_291">90</span></span></label><label><input/><span class="_label_19sud_225 group relative">Digital Health<span class="_count_19sud_291">86</span></span></label><label><input/><span class="_label_19sud_225 group relative">Proptech<span class="_count_19sud_291">84</span></span></label><label><input/><span class="_label_19sud_225 group relative">Sales<span class="_count_19sud_291">83</span></span></label><label><input/><span class="_label_19sud_225 group relative">Gaming<span class="_count_19sud_291">81</span></span></label><label><input/><span class="_label_19sud_225 group relative">Security<span class="_count_19sud_291">74</span></span></label><label><input/><span class="_label_19sud_225 group relative">Video<span class="_count_19sud_291">73</span></span></label><label><input/><span class="_label_19sud_225 group relative">AI Assistant<span class="_count_19sud_291">69</span></span></label><label><input/><span class="_label_19sud_225 group relative">Social<span class="_count_19sud_291">69</span></span></label><label><input/><span class="_label_19sud_225 group relative">Medical Devices<span class="_count_19sud_291">67</span></span></label><label><input/><span class="_label_19sud_225 group relative">Community<span class="_count_19sud_291">64</span></span></label><label><input/><span class="_label_19sud_225 group relative">Real Estate<span class="_count_19sud_291">63</span></span></label><label><input/><span class="_label_19sud_225 group relative">HR Tech<span class="_count_19sud_291">62</span></span></label><label><input/><span class="_label_19sud_225 group relative">Supply Chain<span class="_count_19sud_291">62</span></span></label><label><input/><span class="_label_19sud_225 group relative">Hard Tech<span class="_count_19sud_291">61</span></span></label><label><input/><span class="_label_19sud_225 group relative">Marketing<span class="_count_19sud_291">60</span></span></label><label><input/><span class="_label_19sud_225 group relative">Computer Vision<span class="_count_19sud_291">59</span></span></label><label><input/><span class="_label_19sud_225 group relative">Recruiting<span class="_count_19sud_291">58</span></span></label><label><input/><span class="_label_19sud_225 group relative">Enterprise Software<span class="_count_19sud_291">56</span></span></label><label><input/><span class="_label_19sud_225 group relative">Insurance<span class="_count_19sud_291">56</span></span></label><label><input/><span class="_label_19sud_225 group relative">Design Tools<span class="_count_19sud_291">55</span></span></label><label><input/><span class="_label_19sud_225 group relative">Robotics<span class="_count_19sud_291">55</span></span></label><label><input/><span class="_label_19sud_225 group relative">Delivery<span class="_count_19sud_291">53</span></span></label><label><input/><span class="_label_19sud_225 group relative">Construction<span class="_count_19sud_291">51</span></span></label><label><input/><span class="_label_19sud_225 group relative">Neobank<span class="_count_19sud_291">51</span></span></label><label><input/><span class="_label_19sud_225 group relative">eLearning<span class="_count_19sud_291">51</span></span></label><label><input/><span class="_label_19sud_225 group relative">Investing<span class="_count_19sud_291">50</span></span></label><label><input/><span class="_label_19sud_225 group relative">Infrastructure<span class="_count_19sud_291">49</span></span></label><label><input/><span class="_label_19sud_225 group relative">Finance<span class="_count_19sud_291">47</span></span></label><label><input/><span class="_label_19sud_225 group relative">Food Tech<span class="_count_19sud_291">47</span></span></label><label><input/><span class="_label_19sud_225 group relative">Retail<span class="_count_19sud_291">47</span></span></label><label><input/><span class="_label_19sud_225 group relative">Entertainment<span class="_count_19sud_291">46</span></span></label><label><input/><span class="_label_19sud_225 group relative">Messaging<span class="_count_19sud_291">46</span></span></label><label><input/><span class="_label_19sud_225 group relative">Automation<span class="_count_19sud_291">45</span></span></label><label><input/><span class="_label_19sud_225 group relative">Collaboration<span class="_count_19sud_291">45</span></span></label><label><input/><span class="_label_19sud_225 group relative">Mental Health Tech<span class="_count_19sud_291">45</span></span></label><label><input/><span class="_label_19sud_225 group relative">Compliance<span class="_count_19sud_291">44</span></span></label><label><input/><span class="_label_19sud_225 group relative">IoT<span class="_count_19sud_291">43</span></span></label><label><input/><span class="_label_19sud_225 group relative">Therapeutics<span class="_count_19sud_291">41</span></span></label><label><input/><span class="_label_19sud_225 group relative">Travel<span class="_count_19sud_291">40</span></span></label><label><input/><span class="_label_19sud_225 group relative">LegalTech<span class="_count_19sud_291">39</span></span></label><label><input/><span class="_label_19sud_225 group relative">Media<span class="_count_19sud_291">38</span></span></label><label><input/><span class="_label_19sud_225 group relative">India<span class="_count_19sud_291">37</span></span></label><label><input/><span class="_label_19sud_225 group relative">Deep Learning<span class="_count_19sud_291">35</span></span></label><label><input/><span class="_label_19sud_225 group relative">Subscriptions<span class="_count_19sud_291">35</span></span></label><label><input/><span class="_label_19sud_225 group relative">AI-Enhanced Learning<span class="_count_19sud_291">34</span></span></label><label><input/><span class="_label_19sud_225 group relative">Creator Economy<span class="_count_19sud_291">34</span></span></label><label><input/><span class="_label_19sud_225 group relative">Manufacturing<span class="_count_19sud_291">34</span></span></label><label><input/><span class="_label_19sud_225 group relative">Transportation<span class="_count_19sud_291">34</span></span></label><label><input/><span class="_label_19sud_225 group relative">DevOps<span class="_count_19sud_291">33</span></span></label><label><input/><span class="_label_19sud_225 group relative">Diagnostics<span class="_count_19sud_291">33</span></span></label><label><input/><span class="_label_19sud_225 group relative">ClimateTech<span class="_count_19sud_291">32</span></span></label><label><input/><span class="_label_19sud_225 group relative">Telemedicine<span class="_count_19sud_291">32</span></span></label><label><input/><span class="_label_19sud_225 group relative">DevSecOps<span class="_count_19sud_291">31</span></span></label><label><input/><span class="_label_19sud_225 group relative">GovTech<span class="_count_19sud_291">31</span></span></label><label><input/><span class="_label_19sud_225 group relative">No-code<span class="_count_19sud_291">31</span></span></label><label><input/><span class="_label_19sud_225 group relative">Advertising<span class="_count_19sud_291">30</span></span></label><label><input/><span class="_label_19sud_225 group relative">Cloud Computing<span class="_count_19sud_291">30</span></span></label><label><input/><span class="_label_19sud_225 group relative">Energy<span class="_count_19sud_291">30</span></span></label><label><input/><span class="_label_19sud_225 group relative">Health &amp; Wellness<span class="_count_19sud_291">30</span></span></label><label><input/><span class="_label_19sud_225 group relative">Workflow Automation<span class="_count_19sud_291">30</span></span></label><label><input/><span class="_label_19sud_225 group relative">AI-powered Drug Discovery<span class="_count_19sud_291">29</span></span></label><label><input/><span class="_label_19sud_225 group relative">AIOps<span class="_count_19sud_291">29</span></span></label><label><input/><span class="_label_19sud_225 group relative">Genomics<span class="_count_19sud_291">29</span></span></label><label><input/><span class="_label_19sud_225 group relative">NLP<span class="_count_19sud_291">29</span></span></label><label><input/><span class="_label_19sud_225 group relative">Agriculture<span class="_count_19sud_291">28</span></span></label><label><input/><span class="_label_19sud_225 group relative">Drug discovery<span class="_count_19sud_291">28</span></span></label><label><input/><span class="_label_19sud_225 group relative">Food &amp; Beverage<span class="_count_19sud_291">28</span></span></label><label><input/><span class="_label_19sud_225 group relative">Grocery<span class="_count_19sud_291">28</span></span></label><label><input/><span class="_label_19sud_225 group relative">Banking as a Service<span class="_count_19sud_291">27</span></span></label><label><input/><span class="_label_19sud_225 group relative">Nonprofit<span class="_count_19sud_291">27</span></span></label><label><input/><span class="_label_19sud_225 group relative">Electric Vehicles<span class="_count_19sud_291">26</span></span></label><label><input/><span class="_label_19sud_225 group relative">Cybersecurity<span class="_count_19sud_291">25</span></span></label><label><input/><span class="_label_19sud_225 group relative">Documents<span class="_count_19sud_291">25</span></span></label></div></div><div class="_line_19sud_215"></div><div class="_facet_19sud_85"><h4>Company Size </h4><span class="_label_19sud_225">1 - 1,000+</span><div class="_noUiHorizontal_19sud_85 noUi-target noUi-ltr noUi-horizontal noUi-txt-dir-ltr"><div class="noUi-base"><div class="noUi-connects"><div class="noUi-connect"></div></div><div class="noUi-origin"><div class="noUi-handle noUi-handle-lower"><div class="noUi-touch-area"></div></div></div><div class="noUi-origin"><div class="noUi-handle noUi-handle-upper"><div class="noUi-touch-area"></div></div></div></div></div><div></div></div><div class="_line_19sud_215"></div><div class="_facet_19sud_85"><label><input/><span class="_label_19sud_225">Public Application Video</span><span class="_count_19sud_291">69</span></label></div><div class="_facet_19sud_85"><label><input/><span class="_label_19sud_225">Public Demo Day Video</span><span class="_count_19sud_291">124</span></label></div><div class="_facet_19sud_85"><label><input/><span class="_label_19sud_225">Has Application Answers</span><span class="_count_19sud_291">95</span></label></div><div class="_facet_19sud_85"><label><input/><span class="_label_19sud_225">Has Bonus Questions</span><span class="_count_19sud_291">97</span></label></div><div class="_showResults_19sud_153"><button>Show 1,000+ companies</button></div></div></div><div class="_rightCol_19sud_576"><div class="_section_19sud_146 _searchBox_19sud_186"><input class=""/></div><div class="_showFilters_19sud_150"><button>Filter</button></div><div class="_message_19sud_543">Showing 40 of 1,000+ companies</div><div class="_section_19sud_146 _results_19sud_327"><a class="_company_19sud_339" href="/companies/airbnb"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Airbnb</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">Book accommodations around the world.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W09"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W09</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Consumer"><span class="pill _pill_19sud_33">Consumer</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Travel, Leisure and Tourism"><span class="pill _pill_19sud_33">Travel, Leisure and Tourism</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/instacart"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Instacart</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">Marketplace for grocery delivery and pickup</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S12"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S12</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Consumer"><span class="pill _pill_19sud_33">Consumer</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Food and Beverage"><span class="pill _pill_19sud_33">Food and Beverage</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/doordash"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">DoorDash</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">Restaurant delivery.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S13"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S13</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Consumer"><span class="pill _pill_19sud_33">Consumer</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Food and Beverage"><span class="pill _pill_19sud_33">Food and Beverage</span></a></div></div></div></div></a><a class="_advertContainer_19sud_515" href="/"><div class="_advert_19sud_515"><span><b>Why YC?</b>We give our companies a disproportionate advantage.</span><button>Learn More</button></div></a><a class="_company_19sud_339" href="/companies/coinbase"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Coinbase</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">Buy, sell, and manage cryptocurrencies.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S12"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S12</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Fintech"><span class="pill _pill_19sud_33">Fintech</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Banking and Exchange"><span class="pill _pill_19sud_33">Banking and Exchange</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/dropbox"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Dropbox</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">Backup and share files in the cloud.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S07"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S07</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Productivity"><span class="pill _pill_19sud_33">Productivity</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/gitlab"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">GitLab</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">A complete DevOps platform delivered as a single application.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W15"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W15</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Engineering, Product and Design"><span class="pill _pill_19sud_33">Engineering, Product and Design</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/ginkgo-bioworks"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Ginkgo Bioworks</span><span class="_coLocation_19sud_470">Boston, MA, USA</span></div><div><span class="_coDescription_19sud_479">Our mission is to make biology easier to engineer.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S14"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S14</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Healthcare"><span class="pill _pill_19sud_33">Healthcare</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Industrial Bio"><span class="pill _pill_19sud_33">Industrial Bio</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/pagerduty"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">PagerDuty</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">Real-time visibility into critical apps and services all in one place.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S10"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S10</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Engineering, Product and Design"><span class="pill _pill_19sud_33">Engineering, Product and Design</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/amplitude"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Amplitude</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">Digital Analytics Platform</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W12"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W12</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Analytics"><span class="pill _pill_19sud_33">Analytics</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/matterport"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Matterport</span><span class="_coLocation_19sud_470">Sunnyvale, CA, USA</span></div><div><span class="_coDescription_19sud_479">Turn physical objects and environments into 3D models in seconds.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W12"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W12</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Consumer"><span class="pill _pill_19sud_33">Consumer</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Virtual and Augmented Reality"><span class="pill _pill_19sud_33">Virtual and Augmented Reality</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/weave"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Weave</span><span class="_coLocation_19sud_470">Lehi, UT, USA</span></div><div><span class="_coDescription_19sud_479">Customer communication and payments platform.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W14"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W14</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Office Management"><span class="pill _pill_19sud_33">Office Management</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/notable-labs"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Notable Labs</span><span class="_coLocation_19sud_470">Foster City, CA, USA</span></div><div><span class="_coDescription_19sud_479">Personalized drug discovery for blood cancer.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W15"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W15</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Healthcare"><span class="pill _pill_19sud_33">Healthcare</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Diagnostics"><span class="pill _pill_19sud_33">Diagnostics</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/presto"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Presto</span><span class="_coLocation_19sud_470">San Carlos, CA, USA</span></div><div><span class="_coDescription_19sud_479">Digital meets physical for big chain restaurants</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S10"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S10</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Retail"><span class="pill _pill_19sud_33">Retail</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/rigetti-computing"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Rigetti Computing</span><span class="_coLocation_19sud_470">Berkeley, CA, USA</span></div><div><span class="_coDescription_19sud_479">Quantum coherent supercomputing.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S14"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S14</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Industrials"><span class="pill _pill_19sud_33">Industrials</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Manufacturing and Robotics"><span class="pill _pill_19sud_33">Manufacturing and Robotics</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/pardes-bio"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Pardes Biosciences</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">We break viruses</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S20"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S20</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Healthcare"><span class="pill _pill_19sud_33">Healthcare</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Drug Discovery and Delivery"><span class="pill _pill_19sud_33">Drug Discovery and Delivery</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/embark-trucks"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Embark Trucks</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">Self-driving semi trucks.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W16"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W16</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Industrials"><span class="pill _pill_19sud_33">Industrials</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Automotive"><span class="pill _pill_19sud_33">Automotive</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/momentus"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Momentus</span><span class="_coLocation_19sud_470">Santa Clara, CA, USA</span></div><div><span class="_coDescription_19sud_479">The space infrastructure services company</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S18"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S18</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Industrials"><span class="pill _pill_19sud_33">Industrials</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Aviation and Space"><span class="pill _pill_19sud_33">Aviation and Space</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/lucira-health"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Lucira Health</span><span class="_coLocation_19sud_470">Emeryville, CA, USA</span></div><div><span class="_coDescription_19sud_479">Next generation infectious disease test kits</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W15"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W15</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Healthcare"><span class="pill _pill_19sud_33">Healthcare</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Diagnostics"><span class="pill _pill_19sud_33">Diagnostics</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/stripe"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Stripe</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">Economic infrastructure for the internet.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S09"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S09</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Fintech"><span class="pill _pill_19sud_33">Fintech</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/cruise"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Cruise</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">Self-driving cars.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W14"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W14</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Industrials"><span class="pill _pill_19sud_33">Industrials</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Automotive"><span class="pill _pill_19sud_33">Automotive</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/opensea"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">OpenSea</span><span class="_coLocation_19sud_470">New York, NY, USA</span></div><div><span class="_coDescription_19sud_479">OpenSea is the world’s leading peer-to-peer marketplace for NFTs</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W18"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W18</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Consumer"><span class="pill _pill_19sud_33">Consumer</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Content"><span class="pill _pill_19sud_33">Content</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/faire"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Faire</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">The global online platform empowering independent retail.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W17"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W17</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Retail"><span class="pill _pill_19sud_33">Retail</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/brex"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Brex</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">Business accounts, corporate cards, and spend management software.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W17"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W17</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Fintech"><span class="pill _pill_19sud_33">Fintech</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Banking and Exchange"><span class="pill _pill_19sud_33">Banking and Exchange</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/deel"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Deel</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">The all-in-one HR platform for global teams.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W19"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W19</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Finance and Accounting"><span class="pill _pill_19sud_33">Finance and Accounting</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/rippling"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Rippling</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">One place to run all your HR, IT, and Finance. Globally.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W17"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W17</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Human Resources"><span class="pill _pill_19sud_33">Human Resources</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/reddit"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Reddit</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">The frontpage of the internet.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S05"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S05</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Consumer"><span class="pill _pill_19sud_33">Consumer</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Content"><span class="pill _pill_19sud_33">Content</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/gusto"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Gusto</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">Provides growing businesses with everything to take care of their team</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W12"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W12</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Human Resources"><span class="pill _pill_19sud_33">Human Resources</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/flexport"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Flexport</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">Platform for global logistics.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W14"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W14</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Supply Chain and Logistics"><span class="pill _pill_19sud_33">Supply Chain and Logistics</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/razorpay"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Razorpay</span><span class="_coLocation_19sud_470">Bengaluru, KA, India</span></div><div><span class="_coDescription_19sud_479">India's only full-stack financial solutions company for businesses.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W15"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W15</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Fintech"><span class="pill _pill_19sud_33">Fintech</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Payments"><span class="pill _pill_19sud_33">Payments</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/scale-ai"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Scale AI</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">Data-centric infrastructure to accelerate the development of AI </span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S16"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S16</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Engineering, Product and Design"><span class="pill _pill_19sud_33">Engineering, Product and Design</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/benchling"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Benchling</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">Unlocking the power of biotech with modern software for modern science</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S12"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S12</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/fivetran"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Fivetran</span><span class="_coLocation_19sud_470">Oakland, CA, USA</span></div><div><span class="_coDescription_19sud_479">The leader in automated data movement</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W13"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W13</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Infrastructure"><span class="pill _pill_19sud_33">Infrastructure</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/rappi"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Rappi</span><span class="_coLocation_19sud_470">Bogotá, Bogota, Colombia</span></div><div><span class="_coDescription_19sud_479">On-demand delivery and financial services for Latin America.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W16"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W16</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Consumer"><span class="pill _pill_19sud_33">Consumer</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Food and Beverage"><span class="pill _pill_19sud_33">Food and Beverage</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/checkr"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Checkr</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">People infrastructure for the future of work</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S14"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S14</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Human Resources"><span class="pill _pill_19sud_33">Human Resources</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/zapier"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Zapier</span><span class="_coLocation_19sud_470">Mountain View, CA, USA</span></div><div><span class="_coDescription_19sud_479">The easiest way to automate your work.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S12"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S12</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Productivity"><span class="pill _pill_19sud_33">Productivity</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/meesho"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Meesho</span><span class="_coLocation_19sud_470">Bengaluru, KA, India</span></div><div><span class="_coDescription_19sud_479">Democratizing internet commerce for everyone in India</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S16"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S16</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Retail"><span class="pill _pill_19sud_33">Retail</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/monzo-bank"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Monzo Bank</span><span class="_coLocation_19sud_470">London, England, United Kingdom</span></div><div><span class="_coDescription_19sud_479">A bank as smart as your phone.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?industry=Fintech"><span class="pill _pill_19sud_33">Fintech</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Consumer Finance"><span class="pill _pill_19sud_33">Consumer Finance</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/relativity-space"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Relativity Space</span><span class="_coLocation_19sud_470">Los Angeles, CA, USA</span></div><div><span class="_coDescription_19sud_479">Building humanity’s multiplanetary future.</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=W16"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>W16</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Industrials"><span class="pill _pill_19sud_33">Industrials</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Aviation and Space"><span class="pill _pill_19sud_33">Aviation and Space</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/webflow"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Webflow</span><span class="_coLocation_19sud_470">San Francisco, CA, USA</span></div><div><span class="_coDescription_19sud_479">Professional website design and publishing platform. </span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S13"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S13</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Marketing"><span class="pill _pill_19sud_33">Marketing</span></a></div></div></div></div></a><a class="_company_19sud_339" href="/companies/bird"><div class="relative flex w-full items-center justify-start"><div class="flex w-20 shrink-0 grow-0 basis-20 items-center pr-4"><img class="rounded-full bg-gray-100"/></div><div class="flex flex-1 items-center justify-between"><div class="lg:max-w-[90%]"><div><span class="_coName_19sud_454">Bird</span><span class="_coLocation_19sud_470">Amsterdam, NH, Netherlands</span></div><div><span class="_coDescription_19sud_479">The world’s largest omnichannel communications platform</span></div><div class="_pillWrapper_19sud_33"><a class="_tagLink_19sud_1026" href="/companies?batch=S16"><span class="pill _pill_19sud_33"><svg class="svg-inline--fa fa-y-combinator fa-fw"><path></path></svg>S16</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=B2B"><span class="pill _pill_19sud_33">B2B</span></a><a class="_tagLink_19sud_1026" href="/companies?industry=Engineering, Product and Design"><span class="pill _pill_19sud_33">Engineering, Product and Design</span></a></div></div></div></div></a></div><div>Loading more...</div></div></div></div></section></div></div></body></html>"""

In [19]:
from lxml import html

# Assuming 'html_content' contains your HTML data.
# html_content = "<html><body><p>Hello, World!</p></body></html>"
tree = html.fromstring(html_content)
tree

ParserError: Document is empty